# Vizier Basics
Below, we provide examples of how to:

*   Define a problem statement and study configuration.
*   Setup a local server.
*   Setup a client to connect to the server.
*   Perform a typical tuning loop.
*   Use other client APIs.

## Reference imports

In [ ]:
from vizier.service import clients
from vizier.service import pyvizier as vz
from vizier.service import vizier_service

## Setting up the problem statement

Here we setup the problem statement, which contains information about the search space and the metrics to optimize.

In [ ]:
problem = vz.ProblemStatement()
problem.search_space.root.add_float_param('x', 0.0, 1.0)
problem.search_space.root.add_float_param('y', 0.0, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='maximize_metric', goal=vz.ObjectiveMetricGoal.MAXIMIZE))


def evaluate(x: float, y: float) -> float:
  return x**2 - y**2

## Setting up the study configuration

The study configuration contains additional information, such as the algorithm to use and level of noise that we think the objective will have.

In [ ]:
study_config = vz.StudyConfig.from_problem(problem)
study_config.algorithm = vz.Algorithm.RANDOM_SEARCH

## Setting up the server
As an local example, the server will be run locally here, and will process all client requests. However, it can also be run remotely, to service multiple clients in a distributed manner.


In [ ]:
service = vizier_service.DefaultVizierService()

## Setting up the client
Starts a client, which will connect to the server. Note that the below can be called multiple times, one on each machine, to obtain multiple clients, all working on the same study, especially for tuning jobs which require multiple machines to parallelize the workload.

In [ ]:
clients.environment_variables.service_endpoint = service.endpoint  # Server address.
study_client = clients.Study.from_study_config(
    study_config, owner='owner', study_id='example_study_id')

## Obtaining suggestions
Start requesting suggestions from the server, for evaluating objectives.

In [ ]:
suggestions = study_client.suggest(count=5)
for suggestion in suggestions:
  x = suggestion.parameters['x']
  y = suggestion.parameters['y']
  print('Suggested Parameters (x,y):', x, y)
  final_measurement = vz.Measurement({'maximize_metric': evaluate(x, y)})
  suggestion.complete(final_measurement)

## Find optimal trial
Find the best objective so far, with corresponding suggestion value. For multiobjective cases, there may be multiple outputs of `optimal_trials()`, all corresponding to a Pareto-optimal curve.

In [ ]:
for optimal_trial in study_client.optimal_trials():
  optimal_trial = optimal_trial.materialize()
  print("Optimal Trial Suggestion and Objective:", optimal_trial.parameters,
        optimal_trial.final_measurement)

## Other client commands
The `study` client can also send other requests, such as the following:

In [ ]:
study_client.get_trial(1)  # Get the first trial.
study_client.trials()  # Get all trials so far.

# Obtain only the completed trials.
trial_filter = vz.TrialFilter(status=[vz.TrialStatus.COMPLETED])
study_client.trials(trial_filter=trial_filter)